# Por qué y cómo codificar datos Categóricos Ordinales

## Datos Categoricos
- Nominales: Las categorias que los caracterizan no tienen una cierta relacion de orden
- Ordinales: Las categorias cuentan con una cierta relacion de orden 

No se puede asumir que las categorias son mutuamente equidistantes

## Datos categoricos ordinales: Contexto
#### ¿Que tan satisfecho esta con el servicio?
- Muy insatisfecho
- Insatisfecho
- Neutral
- Satisfecho
- Muy satisfecho

#### ¿Como calificaria la calidad de los alimentos?
- Mala
- Buena
- Muy Buena
- Excelente

In [16]:
import pandas as pd 
import numpy as np

categorias_servicio = ['Muy insatisfecho','Insatisfecho',
                       'Neutral','Satisfecho','Muy satisfecho']

Categorias_calidad = ['Mala','Buena','Muy Buena','Excelente']

encuesta = {
    'servicio': ['Muy insatisfecho','Insatisfecho',
                       'Neutral','Satisfecho','Muy satisfecho', 'Muy insatisfecho'],
    'alimentos':['Mala','Buena','Muy Buena','Excelente','Mala','Buena']
}

# 0: cliente esporadico     1: cliente frecuente
tipo_cliente = [0,0,1,1,0,1]

pd.DataFrame(encuesta)

,servicio,alimentos
0,Muy insatisfecho,Mala
1,Insatisfecho,Buena
2,Neutral,Muy Buena
3,Satisfecho,Excelente
4,Muy satisfecho,Mala
5,Muy insatisfecho,Buena


### Codificador ordinal
Evita la maldicion de la dimensionalidad, pero...

In [17]:
from sklearn.preprocessing import OrdinalEncoder

datos_ord = pd.DataFrame(encuesta)

codificador = OrdinalEncoder(categories=[categorias_servicio,Categorias_calidad])

datos_ord = pd.DataFrame(codificador.fit_transform(datos_ord),
                         columns=['servicio','alimentos'])

datos_ord


,servicio,alimentos
0,0.0,0.0
1,1.0,1.0
2,2.0,2.0
3,3.0,3.0
4,4.0,0.0
5,0.0,1.0


In [18]:
print(datos_ord)
print(codificador.categories_)

   servicio  alimentos
0       0.0        0.0
1       1.0        1.0
2       2.0        2.0
3       3.0        3.0
4       4.0        0.0
5       0.0        1.0
[array(['Muy insatisfecho', 'Insatisfecho', 'Neutral', 'Satisfecho',
       'Muy satisfecho'], dtype=object), array(['Mala', 'Buena', 'Muy Buena', 'Excelente'], dtype=object)]


### Codificador OneHotEncoder
Los potenciales valores los convierte en columnas 

In [19]:
from sklearn.preprocessing import OneHotEncoder

datos_one = pd.DataFrame(encuesta)

codificador = OneHotEncoder()# Retorna una matriz dispersa

print(codificador.fit_transform(datos_one))

print(codificador.fit_transform(datos_one).toarray())

  (0, 1)	1.0
  (0, 7)	1.0
  (1, 0)	1.0
  (1, 5)	1.0
  (2, 3)	1.0
  (2, 8)	1.0
  (3, 4)	1.0
  (3, 6)	1.0
  (4, 2)	1.0
  (4, 7)	1.0
  (5, 1)	1.0
  (5, 5)	1.0
[[0. 1. 0. 0. 0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0. 1. 0. 0. 0.]]


In [20]:
print(np.concatenate(codificador.categories_))

['Insatisfecho' 'Muy insatisfecho' 'Muy satisfecho' 'Neutral' 'Satisfecho'
 'Buena' 'Excelente' 'Mala' 'Muy Buena']


In [21]:
datos_one = pd.DataFrame(codificador.fit_transform(datos_one).toarray(),
                        columns=np.concatenate(codificador.categories_))

datos_one

,Insatisfecho,Muy insatisfecho,Muy satisfecho,Neutral,Satisfecho,Buena,Excelente,Mala,Muy Buena
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
3,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
5,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


### Comparacion de las codificaciones 
Calsificador: Regresion logistica

In [22]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler

print("\n*** Datos Escalados de la Codificación Ordinal")
escalador = MinMaxScaler()
print(datos_ord)
datos_ord = escalador.fit_transform(datos_ord)
print(datos_ord)

print("\n*** Clasificación con Datos codificados con OrdinalEncoder")
modelo = LogisticRegression().fit(datos_ord, tipo_cliente)
print("Predicciones:", modelo.predict(datos_ord))
print("Clases correctas:", tipo_cliente)
print(modelo.predict_proba(datos_ord))

print("\n*** Clasificación con Datos codificados con OneHotEncoder")
modelo = LogisticRegression().fit(datos_one, tipo_cliente)
print("Predicciones:", modelo.predict(datos_one))
print("Clases correctas:", tipo_cliente)
print(modelo.predict_proba(datos_one))


*** Datos Escalados de la Codificación Ordinal
   servicio  alimentos
0       0.0        0.0
1       1.0        1.0
2       2.0        2.0
3       3.0        3.0
4       4.0        0.0
5       0.0        1.0
[[0.         0.        ]
 [0.25       0.33333333]
 [0.5        0.66666667]
 [0.75       1.        ]
 [1.         0.        ]
 [0.         0.33333333]]

*** Clasificación con Datos codificados con OrdinalEncoder
Predicciones: [0 0 1 1 0 0]
Clases correctas: [0, 0, 1, 1, 0, 1]
[[0.56489598 0.43510402]
 [0.50845928 0.49154072]
 [0.45180614 0.54819386]
 [0.39637512 0.60362488]
 [0.57175647 0.42824353]
 [0.50671189 0.49328811]]

*** Clasificación con Datos codificados con OneHotEncoder
Predicciones: [0 0 1 1 0 1]
Clases correctas: [0, 0, 1, 1, 0, 1]
[[0.62301432 0.37698568]
 [0.57519021 0.42480979]
 [0.32279763 0.67720237]
 [0.32279763 0.67720237]
 [0.70505958 0.29494042]
 [0.45116842 0.54883158]]
